## Python, SQL & AI

Python - the powerhouse coding language in the field of data science - offers unparalleled versatility for tasks ranging from data manipulation to predictive modeling. However, it not the only language of significance in this domain. SQL, a widely-used query language, plays a crucial role in extracting and managing structured data, enabling data professionals to interact effectively with databases. Many companies rely heavily on SQL for their day-to-day operations, making it a staple in the toolkit of data analysts and scientists.

Currently, the rise of Generative AI (GenAI) has introduced exciting possibilities, including the automated generation of code to produce specific outputs. This in turn allows for enhanced productivity of data analysts and scientists and opens up innovative avenues for problem-solving. It is the aim of this notebook, and by proxy this project, to seamlessly combine the interactivity of Python (with tools like Pandas), SQL (via SQLite), and GenAI (like GPT-4) to analyze and derive insights from a real-world movie dataset. This unified approach showcases the power of integration, offering a practical and efficient environment for data science workflow.

*24/1/25 - 28/1/25*

In [1]:
import sqlite3
import pandas as pd
from sqlalchemy.orm import sessionmaker
from langchain_openai import ChatOpenAI # note that an OpenAI account and API key is required for the latter half of this code
from langchain.prompts import PromptTemplate
from sqlalchemy import create_engine, MetaData
# if libraries have not been installed, please pip install the requirements.txt file

In [2]:
# Dataset src: https://www.kaggle.com/datasets/shahjhanalam/movie-data-analytics-dataset?select=movie.sqlite
# the file has been downloaded for reference
sqlite_path = "movie.sqlite"
conn = sqlite3.connect(sqlite_path)

table_names = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table'",conn)
# Queries in a SQL-based syntax (particularly for SQLite) can be written directly in the above pandas method to produce a dataframe
# The aim of the above query is to select all of the table names in the database
table_names

,name
0,IMDB
1,earning
2,genre


### Querying Tables

In [3]:
imdb = pd.read_sql_query("SELECT * FROM IMDB",conn)
imdb

,Movie_id,Title,Rating,TotalVotes,MetaCritic,Budget,Runtime,CVotes10,CVotes09,CVotes08,...,Votes3044,Votes3044M,Votes3044F,Votes45A,Votes45AM,Votes45AF,VotesIMDB,Votes1000,VotesUS,VotesnUS
0,36809,12 Years a Slave (2013),8.1,496092,96,20000000.0,134 min,75556,126223,161460,...,8.0,7.9,8.0,7.8,7.8,8.1,8.0,7.7,8.3,8.0
1,30114,127 Hours (2010),7.6,297075,82,18000000.0,94 min,28939,44110,98845,...,7.5,7.5,7.5,7.3,7.3,7.5,7.6,7.0,7.7,7.6
2,37367,50/50 (2011),7.7,283935,72,8000000.0,100 min,28304,47501,99524,...,7.6,7.6,7.6,7.4,7.4,7.5,7.4,7.0,7.9,7.6
3,49473,About Time (2013),7.8,225412,,12000000.0,123 min,38556,43170,70850,...,7.6,7.6,7.7,7.6,7.5,7.8,7.7,6.9,7.8,7.7
4,14867,Amour (2012),7.9,76121,94,8900000.0,127 min,11093,15944,22942,...,7.7,7.7,7.9,7.9,7.8,8.1,6.6,7.2,7.9,7.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,38262,Whiplash (2014),8.5,492285,88,3300000.0,107 min,110404,161864,132656,...,8.3,8.3,8.2,8.1,8.1,8.2,8.7,8.0,8.6,8.4
113,10023,Wreck-It Ralph (2012),7.7,295125,72,165000000.0,,41980,50262,96477,...,7.6,7.6,7.7,7.4,7.4,7.5,7.4,7.2,7.9,7.6
114,27783,X-Men: Days of Future Past (2014),8.0,560736,74,200000000.0,132 min,91765,127521,183578,...,7.8,7.8,8.0,7.7,7.7,7.9,7.5,7.4,8.1,7.9
115,42294,X-Men: First Class (2011),7.8,556713,65,160000000.0,132 min,64428,96219,200144,...,7.7,7.6,7.8,7.6,7.5,7.7,7.6,7.3,7.8,7.7


In [4]:
earn = pd.read_sql_query("SELECT * FROM earning LIMIT 5",conn)
# akin to the head function in pandas, the above query displays the first 5 rows of the earning table
earn # the results are then saved in a dataframe format pandas is used to

,Movie_id,Domestic,Worldwide
0,36809,56671993,187733202.0
1,30114,18335230,60738797.0
2,37367,35014192,39187783.0
3,49473,15322921,87100449.0
4,14867,6739492,19839492.0


In [5]:
genre = pd.read_sql_query("SELECT * FROM genre",conn)
genre # unlike the initial IMDB table with 117 rows, the genre table appears to have significantly more rows
# while being relted to IMDB via the Movie_id column. This implies the existence of duplicates/ new data in the 
# genre table - which shall be explored in the code below

,Movie_id,genre
0,36809,Biography
1,30114,Adventure
2,37367,Comedy
3,49473,Comedy
4,14867,Drama
...,...,...
346,38262,
347,10023,Comedy
348,27783,Sci-Fi
349,42294,Sci-Fi


In [6]:
genre["genre"].value_counts() 
# with the information thrown into a dataframe, methods in pandas can be utilised to observe the elements inside
# it quickly and efficiently (especially when compared to SQL queries). A similar result will be explored later on 
# using SQL, but this definitely highlights the inclusiveness of Python in data science as a language with syntax
# that is quick and easy to write and run

genre
Drama        77
Adventure    43
             41
Action       33
Comedy       31
Biography    21
Sci-Fi       18
Thriller     14
Romance      13
Animation    13
Crime        11
Mystery       7
Fantasy       7
History       6
Sport         3
Family        3
Music         3
Western       2
War           2
Horror        2
Musical       1
Name: count, dtype: int64

In [7]:
list(filter(lambda x:x.strip() == '',genre["genre"].unique()))

['']

### Joining & Filtering Tables

In [8]:
nf = pd.read_sql_query("SELECT * FROM IMDB INNER JOIN GENRE ON IMDB.Movie_id = GENRE.Movie_id",conn)
nf[nf["genre"]==''] 

# joining the IMDB table w the GENRE table, we now can query those movies with missing data
# while the latter part of the above comment has been done using pandas, it is also fairly easy to do in SQL and 
# will be observed further down

,Movie_id,Title,Rating,TotalVotes,MetaCritic,Budget,Runtime,CVotes10,CVotes09,CVotes08,...,Votes3044F,Votes45A,Votes45AM,Votes45AF,VotesIMDB,Votes1000,VotesUS,VotesnUS,Movie_id,genre
12,14867,Amour (2012),7.9,76121,94,8900000.0,127 min,11093,15944,22942,...,7.9,7.9,7.8,8.1,6.6,7.2,7.9,7.8,14867,
21,25398,Before Midnight (2013),7.9,106553,94,3000000.0,109 min,16953,22109,31439,...,7.6,7.3,7.4,7.2,8.5,7.0,8.0,7.9,25398,
27,25281,Birdman or (The Unexpected Virtue of Ignorance...,7.8,448725,88,18000000.0,119 min,60209,94476,121637,...,7.3,7.2,7.3,7.0,7.8,7.1,7.9,7.7,25281,
30,43358,Black Swan (2010),8.0,587893,79,13000000.0,108 min,93798,136615,174500,...,8.0,7.5,7.5,7.4,7.9,7.6,8.0,8.0,43358,
33,36798,Boyhood (2014),7.9,290327,100,4000000.0,165 min,49673,62055,76838,...,7.6,7.7,7.7,7.7,8.2,7.2,8.0,7.9,36798,
34,36798,Boyhood (2014),7.9,290327,100,4000000.0,165 min,49673,62055,76838,...,7.6,7.7,7.7,7.7,8.2,7.2,8.0,7.9,36798,
45,31370,Captain Fantastic (2016),7.9,115194,72,5000000.0,,16165,24762,39686,...,7.9,7.7,7.6,7.9,8.5,6.8,7.8,7.8,31370,
51,38131,Creed (2015),7.6,179795,82,35000000.0,133 min,21364,28964,58237,...,7.4,7.3,7.3,7.5,7.8,7.1,7.9,7.5,38131,
54,34191,Dallas Buyers Club (2013),8.0,357641,84,5000000.0,117 min,37544,82276,145488,...,8.0,7.8,7.8,8.0,7.9,7.2,8.0,7.9,34191,
66,43526,Detachment (2011),7.7,62352,,,,8602,11683,20411,...,7.7,7.5,7.5,7.7,6.7,6.5,7.5,7.7,43526,


In [9]:
pd.read_sql_query("SELECT * FROM GENRE GROUP BY Movie_id",conn)

# when grouped by the Movie_id col, we can see that the number of rows in the 
# GENRE table is the same as that of the IMDB table - implying that there is likely not
# any new movies in the GENRE table (though this can be confirmed w a full outer JOIN). This likely means 
# that each movie is assigned 1/more genres - and a count of these genres per movie can be done as shown below

,Movie_id,genre
0,10015,Drama
1,10023,Animation
2,10099,Animation
3,10454,Action
4,10617,Drama
...,...,...
112,48554,Biography
113,48626,Biography
114,49473,Comedy
115,49518,Comedy


In [10]:
pd.read_sql_query("SELECT Movie_id,COUNT(genre) AS COUNT FROM GENRE WHERE GENRE!='' GROUP BY Movie_id ORDER BY Count DESC",conn)

# The Count of the number of genres for each movie_id (barring those with the empty genre '') is shown below
# we can see from the rough result that since the dataframe has been ordered, there aren't any movies w less than 1 NAMED genre
# assigned to them and none with more than 3 genres. Such information can prove useful in network analysis to observe how
# genres are related to each other - for example we can expect a result like having a lot of movies that are labelled w the "Thriller"
# genre to also be labelled as "Drama" films - which we can confirm in the database using SQL as shown below!

,Movie_id,COUNT
0,49590,3
1,49518,3
2,49473,3
3,48626,3
4,48554,3
...,...,...
112,34429,1
113,30154,1
114,29611,1
115,22767,1


In [11]:
pd.read_sql_query("SELECT Movie_id,GROUP_CONCAT(genre, ', ') as genres FROM GENRE WHERE GENRE!='' GROUP BY Movie_id",conn)

# the observation of the strength of the interrelationships between genres 
# can be done using the TransactionEncoder for apriori analysis

,Movie_id,genres
0,10015,"Drama, Thriller"
1,10023,"Animation, Adventure, Comedy"
2,10099,"Animation, Action, Adventure"
3,10454,"Action, Sci-Fi"
4,10617,"Drama, History, Thriller"
...,...,...
112,48554,"Biography, Drama, Sport"
113,48626,"Biography, Drama, Thriller"
114,49473,"Comedy, Drama, Fantasy"
115,49518,"Comedy, Drama, Romance"


## Feature Engineering

In [12]:
pd.read_sql_query("""
    SELECT E.Movie_id, E.DOMESTIC, E.WORLDWIDE, I.TITLE, I.RATING, I.TOTALVOTES, I.BUDGET, I.RUNTIME,
    (DOMESTIC/ WORLDWIDE)*100 AS Perc_Domestic FROM EARNING AS E 
    JOIN IMDB AS I ON E.Movie_id = I.Movie_id  
    WHERE Perc_Domestic > 50 ORDER BY Perc_Domestic DESC""",
    conn)

# SQL can even be used for the creation/ showcasing of new columns in tables - in this case the percentage of a 
# movie's domestic earnings on the global stage has been assigned to the Perc_Domestic column, with the value of 100
# meaning that the movie was likely distributed only in its country of origin/ highly popular there. This can be tallied 
# with further queries such as the budget/ votes of a movie to understand its popularity/ the popularity of films in a certain
# genre abroad and even at home

,Movie_id,Domestic,Worldwide,Title,Rating,TotalVotes,Budget,Runtime,Perc_Domestic
0,31370,5879736,5.879736e+06,Captain Fantastic (2016),7.9,115194,5000000.0,,100.000000
1,43526,72689,7.268900e+04,Detachment (2011),7.7,62352,,,100.000000
2,10015,1436900,1.436900e+06,Disconnect (2012),7.6,65448,10000000.0,115 min,100.000000
3,49518,1755212,1.755212e+06,Flipped (2010),7.7,66461,14000000.0,,100.000000
4,23963,5205468,5.205468e+06,Hunt for the Wilderpeople (2016),7.9,56964,2500000.0,,100.000000
5,29611,260354,2.603540e+05,"I, Daniel Blake (2016)",7.9,28350,,,100.000000
6,30154,1013100,1.013100e+06,Short Term 12 (2013),8.0,63459,1000000.0,,100.000000
7,20709,22321,2.232100e+04,Tyrannosaur (2011),7.6,26016,1000000.0,,100.000000
8,37367,35014192,3.918778e+07,50/50 (2011),7.7,283935,8000000.0,100 min,89.349765
9,21508,41003371,4.812638e+07,End of Watch (2012),7.7,194675,7000000.0,109 min,85.199360


In [13]:
pd.read_sql_query("""
    SELECT E.Movie_id, E.DOMESTIC, E.WORLDWIDE, I.TITLE, I.RATING, I.TOTALVOTES, I.BUDGET, I.RUNTIME,
    (DOMESTIC/ WORLDWIDE)*100 AS Perc_Domestic FROM EARNING AS E 
    JOIN IMDB AS I ON E.Movie_id = I.Movie_id  
    WHERE Perc_Domestic < 50 ORDER BY Perc_Domestic DESC, BUDGET DESC""",conn)

# similar as the query above - just that the filtering clause is to show those movies where less than 50% of its earnings
# were domestic. We can see that a lot more of the movies in our table fall into this category. 

,Movie_id,Domestic,Worldwide,Title,Rating,TotalVotes,Budget,Runtime,Perc_Domestic
0,45231,48071303,96188903.0,Kick-Ass (2010),7.7,461860,30000000.0,117 min,49.975934
1,37979,61002302,122126687.0,Prisoners (2013),8.1,441037,46000000.0,153 min,49.950018
2,34191,27298285,55198285.0,Dallas Buyers Club (2013),8.0,357641,5000000.0,117 min,49.454951
3,31722,100546139,203388186.0,Arrival (2016),8.0,370842,47000000.0,116 min,49.435585
4,41963,424668047,865011746.0,The Hunger Games: Catching Fire (2013),7.6,531466,130000000.0,146 min,49.093905
...,...,...,...,...,...,...,...,...,...
71,41048,3237118,13624522.0,Sing Street (2016),8.0,56066,4000000.0,,23.759498
72,12744,1446634,7523634.0,Pride (2014),7.8,40577,,,19.227862
73,49473,15322921,87100449.0,About Time (2013),7.8,225412,12000000.0,123 min,17.592241
74,20420,223838,4749516.0,Tucker and Dale vs Evil (2010),7.6,138624,5000000.0,,4.712859


In [14]:
pd.read_sql_query("""
    SELECT SUBSTR(Title, INSTR(Title, '(') + 1, INSTR(Title, ')') - INSTR(Title, '(') - 1) 
    AS YEAR, COUNT(*) AS Count FROM IMDB
    GROUP BY YEAR HAVING YEAR LIKE '2___' 
    ORDER BY YEAR DESC""",conn)

# It is noticeable that the Title column appears to have the year in which the movie was released
# in brackets beside the name of the movie. This can be a useful feature to isolate and append to either
# a separate column/ to group the existing data together by - the latter instance of which has been executed.
# Note that a 'Having' clause has been implemented to ensure that only data which looks like a Year will be considered

,YEAR,Count
0,2016,18
1,2015,13
2,2014,22
3,2013,18
4,2012,14
5,2011,13
6,2010,18


In [15]:
pd.read_sql_query("""
    SELECT SUBSTR(RUNTIME, 1, INSTR(RUNTIME,' min')) AS Duration_min,* FROM IMDB
    WHERE TRIM(Duration_min) != ''
    ORDER BY CAST(Duration_min AS INTEGER) DESC
""",conn)

# It is also noteworthy that the Runtime column appears to have the suffix "min" in it, which can prove complex
# for mathematical operations to be performed on it. The Duration_min column has thus been created by removing this
# suffix and being cast as an Integer type for the ease of numeric operations to be carried out

,Duration_min,Movie_id,Title,Rating,TotalVotes,MetaCritic,Budget,Runtime,CVotes10,CVotes09,...,Votes3044,Votes3044M,Votes3044F,Votes45A,Votes45AM,Votes45AF,VotesIMDB,Votes1000,VotesUS,VotesnUS
0,180,35004,The Wolf of Wall Street (2013),8.2,881984,75,100000000.0,180 min,171660,236650,...,8.0,8.1,7.7,7.6,7.6,7.5,7.8,7.8,8.1,8.1
1,169,46900,Interstellar (2014),8.6,1075163,74,165000000.0,169 min,394006,291172,...,8.4,8.4,8.2,7.9,7.9,7.7,8.2,7.9,8.4,8.5
2,169,41835,The Hobbit: An Unexpected Journey (2012),7.9,675678,,180000000.0,169 min,135848,132202,...,7.7,7.7,7.9,7.7,7.6,8.0,7.0,7.5,7.8,7.8
3,165,36798,Boyhood (2014),7.9,290327,100,4000000.0,165 min,49673,62055,...,7.8,7.8,7.6,7.7,7.7,7.7,8.2,7.2,8.0,7.9
4,165,11971,Django Unchained (2012),8.4,1056822,81,100000000.0,165 min,234824,339329,...,8.3,8.3,8.3,8.0,8.0,8.1,8.0,7.8,8.4,8.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,98,23417,Philomena (2013),7.6,81855,77,12000000.0,98 min,6420,12446,...,7.6,7.5,7.7,7.8,7.7,7.9,7.9,7.1,7.7,7.6
92,95,42732,Inside Out (2015),8.2,430229,94,175000000.0,95 min,87509,113244,...,8.1,8.1,8.1,7.9,7.9,7.9,8.6,7.6,8.2,8.1
93,94,30114,127 Hours (2010),7.6,297075,82,18000000.0,94 min,28939,44110,...,7.5,7.5,7.5,7.3,7.3,7.5,7.6,7.0,7.7,7.6
94,94,29635,Moonrise Kingdom (2012),7.8,258231,84,16000000.0,94 min,34789,53660,...,7.6,7.6,7.7,7.5,7.5,7.6,7.7,7.2,8.0,7.7


## GenAI & SQLite

Of course, it is essential for any data scientist to master the coding language he uses - whether that be Python, SQL, R, MATLAB etc. But no one person is truly infallible, and there can be instances where incorporating AI into a project can prove beneficial in correcting and clarifying particular code blocks or even providing varying ideas on how to gain insights into an existing dataset. In this section, I utilise GPT-4 in accordance with the above directives to showcase how, as annoying as it admittedly is to myself personally, AI can prove to be very helpful - especially in the years to come.

In [16]:
api = None
open_ai_key_path = "OpenAI_API_SQL.txt" # my API key for OpenAI (saved locally to my computer)
# I did attempt to experiment with other open source models on hugging-face like Gemma and Llama, but OpenAI really takes the cake here
with open(open_ai_key_path,'r') as f:
    api = f.readline()

In [17]:
engine = create_engine(f'sqlite:///{sqlite_path}', echo=True)
Session = sessionmaker(bind=engine)
session = Session()

metadata = MetaData()
metadata.reflect(bind=engine)
schema = "\n".join([f"Table {table.name} columns: {[col.name for col in table.columns]}" for table in metadata.tables.values()])
# the metadata of my database (in terms of the table names and colnames) are retrieved and placed into a schema for the LLM to have
# access to in order to form accurate prompts based on my existing data set

2025-01-28 11:49:16,698 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-01-28 11:49:16,700 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2025-01-28 11:49:16,703 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-01-28 11:49:16,709 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_temp_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2025-01-28 11:49:16,711 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-01-28 11:49:16,715 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("IMDB")
2025-01-28 11:49:16,717 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-01-28 11:49:16,721 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("earning")
2025-01-28 11:49:16,721 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-01-28 11:49:16,728 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("genre")
2025-01-28 11:49:16,733 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-01-2

In [18]:
llm = ChatOpenAI(model='gpt-4',temperature=0, api_key = api)
# temperature is 0 for reproducibility of code (can change a little bit but i advise to alter the prompt instead)
def input_qn(question):
    prompt_template = f"""You are a helpful assistant who converts a user's natural language question into an SQL query for a SQLite database.
    Here is the schema for the database:
    {schema}

    Using the above information given, note that the user asks: "{question}"
    Generate an appropriate SQL query (that is SQLite appropriate) to retrieve the data from the database to answer the user's query.
    """

    prompt = PromptTemplate(input_variables=["schema", "question"],template=prompt_template)
    return prompt, question

In [19]:
input = "If i join the IMDB and GENRE databases together using the movie_id columns, how can i use SQLite to get those rows with empty genres"
prompt, query = input_qn(input)
llm_chain = prompt | llm

# for testing purposes, a simple prompt is given that should yield code that, when executed, provides a known output 
# (Notably the output shld be the same as that in cell 8!)

In [20]:
res = llm_chain.invoke({"schema":schema, "question":query}).content
print(res)

The SQL query to retrieve the data from the database to answer the user's query would be:

```sql
SELECT *
FROM IMDB
JOIN genre ON IMDB.Movie_id = genre.Movie_id
WHERE genre.genre IS NULL OR genre.genre = '';
```


In [21]:
pd.read_sql_query("SELECT * FROM IMDB JOIN GENRE ON IMDB.Movie_id = GENRE.Movie_id WHERE GENRE.genre IS NULL OR GENRE.genre = '';",conn)
# the code generated by AI is more or less placed directly for execution 
# (the IS NULL part is unnecessary for this dataset but can be included)

,Movie_id,Title,Rating,TotalVotes,MetaCritic,Budget,Runtime,CVotes10,CVotes09,CVotes08,...,Votes3044F,Votes45A,Votes45AM,Votes45AF,VotesIMDB,Votes1000,VotesUS,VotesnUS,Movie_id,genre
0,36798,Boyhood (2014),7.9,290327,100,4000000.0,165 min,49673,62055,76838,...,7.6,7.7,7.7,7.7,8.2,7.2,8.0,7.9,36798,
1,43526,Detachment (2011),7.7,62352,,,,8602,11683,20411,...,7.7,7.5,7.5,7.7,6.7,6.5,7.5,7.7,43526,
2,29611,"I, Daniel Blake (2016)",7.9,28350,78,,,3576,5803,9651,...,8.1,8.0,8.0,8.2,9.1,6.9,7.6,7.9,29611,
3,22767,Manchester by the Sea (2016),7.9,150266,96,8500000.0,137 min,18191,33532,46596,...,7.7,7.6,7.6,7.6,8.0,7.1,7.9,7.8,22767,
4,30154,Short Term 12 (2013),8.0,63459,82,1000000.0,,9518,14786,21228,...,7.9,7.7,7.7,8.0,8.5,7.0,8.3,7.9,30154,
5,34429,The Help (2011),8.1,347299,62,25000000.0,146 min,60811,82560,115917,...,8.3,8.0,7.9,8.3,7.9,7.0,8.0,8.0,34429,
6,20709,Tyrannosaur (2011),7.6,26016,65,1000000.0,,2060,4083,9078,...,7.5,7.5,7.4,5.8,6.5,7.4,7.6,,20709,
7,14867,Amour (2012),7.9,76121,94,8900000.0,127 min,11093,15944,22942,...,7.9,7.9,7.8,8.1,6.6,7.2,7.9,7.8,14867,
8,25398,Before Midnight (2013),7.9,106553,94,3000000.0,109 min,16953,22109,31439,...,7.6,7.3,7.4,7.2,8.5,7.0,8.0,7.9,25398,
9,25281,Birdman or (The Unexpected Virtue of Ignorance...,7.8,448725,88,18000000.0,119 min,60209,94476,121637,...,7.3,7.2,7.3,7.0,7.8,7.1,7.9,7.7,25281,


In [22]:
# GPT has thus proven to manage simple sql queries well, but what about more advanced ones with no easily referenceable functions available?

input = """How can I get the interquartile values (25th percentile, median and 50th percentile) 
of my ratings of movies in IMDB without any special PERCENT/ PERCENTILE keywords specifically in SQLITE?"""
prompt, query = input_qn(input)
llm_chain = prompt | llm

In [23]:
res = llm_chain.invoke({"schema":schema, "question":query}).content
print(res)

# Astounding! The generated code actually works (as shown in the cell below). It should be noted however that this is input dependent
# as prior to the conditions of not having "any special PERCENT/ PERCENTILE" keywords being present in the input - the output did yield
# code containing this keywords (likely as while this keyword is reserved in other SQL-like languages, this isn't the case for SQLite)
# This in turn highlights the need to construct accurate, relevant prompts that handle edge cases well in order to yield results that 
# one expects - in other words, we can't just use AI willy - nilly.

SQLite does not have built-in functions for calculating percentiles. However, you can calculate the 25th percentile, median (50th percentile), and 75th percentile using a combination of sorting and counting. Here is an example SQL query:

```sql
SELECT 
    (SELECT MAX(Rating) FROM 
        (SELECT Rating FROM IMDB ORDER BY Rating LIMIT 25 OFFSET 
            (SELECT (COUNT(*) * 25 / 100) FROM IMDB))) AS '25th Percentile',
    (SELECT MAX(Rating) FROM 
        (SELECT Rating FROM IMDB ORDER BY Rating LIMIT 50 OFFSET 
            (SELECT (COUNT(*) * 50 / 100) FROM IMDB))) AS 'Median',
    (SELECT MAX(Rating) FROM 
        (SELECT Rating FROM IMDB ORDER BY Rating LIMIT 75 OFFSET 
            (SELECT (COUNT(*) * 75 / 100) FROM IMDB))) AS '75th Percentile'
FROM IMDB
LIMIT 1;
```

This query calculates the 25th, 50th, and 75th percentiles by sorting the 'Rating' column in ascending order and then selecting the value at the index that corresponds to the desired percentile. The 'LIMIT' clause

In [24]:
# getting IQR range

pd.read_sql_query("""
SELECT 
    (SELECT MAX(Rating) FROM 
        (SELECT Rating FROM IMDB ORDER BY Rating LIMIT 25*(SELECT COUNT(*) FROM IMDB)/100 - 1, 1)) AS '25th Percentile',
    (SELECT AVG(Rating) FROM 
        (SELECT Rating FROM IMDB ORDER BY Rating LIMIT 2*(SELECT COUNT(*) FROM IMDB)/2 - 1, 2)) AS 'Median',
    (SELECT MIN(Rating) FROM 
        (SELECT Rating FROM IMDB ORDER BY Rating DESC LIMIT 25*(SELECT COUNT(*) FROM IMDB)/100 - 1, 1)) AS '75th Percentile'
FROM IMDB
LIMIT 1;
""",conn)

,25th Percentile,Median,75th Percentile
0,7.7,8.8,8


In [25]:
imdb["Rating"].describe() 

# the above result can be confirmed using pandas --> again illustrating the usefulness of analysing data in different languages

count    117.000000
mean       7.873504
std        0.237586
min        7.500000
25%        7.700000
50%        7.800000
75%        8.000000
max        8.800000
Name: Rating, dtype: float64

In [26]:
input = "How can I alter the runtime column in IMDB to NOT have the 'min' suffix (in SQLITE)"
prompt, query = input_qn(input)
llm_chain = prompt | llm

# Indeed, AI can even be used to alter tables directly, rather than simply selecting feature-engineered columns 

In [27]:
res = llm_chain.invoke({"schema":schema, "question":query}).content
print(res)

SQLite does not support the ALTER COLUMN command to modify the data type or format of a column directly. However, you can create a new column, copy the modified data into the new column, and then remove the old column.

Here is a series of SQL commands that will achieve this:

```sql
-- Add a new column
ALTER TABLE IMDB ADD COLUMN 'Runtime_new';

-- Update the new column with the modified data
UPDATE IMDB SET Runtime_new = REPLACE(Runtime, ' min', '');

-- Drop the old column
-- SQLite does not support the DROP COLUMN command directly, so we need to create a new table without the old column and replace the old table with the new one
CREATE TABLE IMDB_new AS SELECT Movie_id, Title, Rating, TotalVotes, MetaCritic, Budget, Runtime_new AS Runtime, CVotes10, CVotes09, CVotes08, CVotes07, CVotes06, CVotes05, CVotes04, CVotes03, CVotes02, CVotes01, CVotesMale, CVotesFemale, CVotesU18, CVotesU18M, CVotesU18F, CVotes1829, CVotes1829M, CVotes1829F, CVotes3044, CVotes3044M, CVotes3044F, CVotes45A

In [28]:
# Most pertinently, AI can be used for the general ideation on the relationship between columns and various ways to query a particular database
# to gain insights into what the data is about and how that can prove useful to a data scientist.

input = "Give me 3 SQL queries to gain as much information from these tables as possible (for SQLITE)"
prompt, query = input_qn(input)
llm_chain = prompt | llm

In [29]:
res = llm_chain.invoke({"schema":schema, "question":query}).content
print(res)

Sure, here are three SQL queries that can provide a lot of information from these tables:

1. This query will give you a list of all movies, their ratings, total votes, and their earnings (both domestic and worldwide):

```sql
SELECT IMDB.Title, IMDB.Rating, IMDB.TotalVotes, earning.Domestic, earning.Worldwide
FROM IMDB
JOIN earning ON IMDB.Movie_id = earning.Movie_id;
```

2. This query will give you a list of all movies, their genres, and their MetaCritic scores:

```sql
SELECT IMDB.Title, genre.genre, IMDB.MetaCritic
FROM IMDB
JOIN genre ON IMDB.Movie_id = genre.Movie_id;
```

3. This query will give you a list of all movies, their budgets, runtimes, and the number of votes they received from males and females:

```sql
SELECT IMDB.Title, IMDB.Budget, IMDB.Runtime, IMDB.CVotesMale, IMDB.CVotesFemale
FROM IMDB;
```

These queries should give you a good amount of information about the movies in the database.


In [30]:
conn.close()

## Conclusion

By no means are API calls to GPT perfect - depending on the prompt they can take a while to output code, code which ultimately could be typed out by any confident data scientist fairly quickly to a high degree of precision. Indeed, I believe that in this project I have demonstrated my knowledge of SQL fairly well and yielded outputs which can be further utilised for Data Visualisation/ Machine Learning purposes. 

Yet this is not to say that AI does not have a place in a data science project. It can render assistance to the data scientist by providing code to clean up data more efficiently, and also offer new ways in which a database can be analysed that a person may not necessarily realise at a particular point in time. I have learnt quite a lot from this project on the interactivity between different coding languages, as well as the use case of AI, and hope that this proves to be useful in a workplace environment to come.